In [1]:
import pandas as pd
import numpy

In [2]:
import gensim

# Load pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load("300features-2window-40mincount")

c:\python27\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
train = open("wiki_texts_tokenized.txt", "r")

In [ ]:
train_text=train.readlines()
print len(train_text)
#print train_text[0]
##Seen lots of noises in the sentences.

# Preprocessing

## 1. Remove repeating Pound Sign

In [ ]:
import re

In [ ]:
train_text=[re.sub('#*#','#',line) for line in train_text]

In [ ]:
print train_text[0]

## 2. Remove English Chars

In [ ]:
train_text=[re.sub('#[a-zA-Z]* ','',line) for line in train_text]

In [ ]:
print train_text[2]

## 3. Clean tags

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
example1 = bs(train["review"][0])
print example1.get_text() #Remove all tabs and markups

## 4. Clean numbers

In [ ]:
letters_only = re.sub("[^a-zA-Z]"," ",example1.get_text() )  # The text to search
print letters_only

## 5. Uniform lower and upper cases

In [ ]:
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

In [ ]:
word_count_dic = {x:words.count(x) for x in words}

## 6. Stopwords

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords
print stopwords.words("english") 

## 4. Compile into a function

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def article_to_sentences(article, tokenizer, remove_stopwords=False):
    sentences=[]
    rev=bs(article.strip()).get_text()
    letter_only=re.sub("[^a-zA-Z]"," ",rev)
    lower_case=letter_only.lower()
    raw_sentences=tokenizer.tokenize(lower_case)
    for sentence in raw_sentences:
        if len(sentence)>1:
            words=sentence.split()
            if remove_stopwords:
                words=[x for x in words if x not in stopwords.words('english')]
            sentences.append(words)
    return sentences

# Train model

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

from gensim.models import word2vec

c:\python27\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-11-30 12:44:52,348 : INFO : 'pattern' package not found; tag filters are not available for English


In [ ]:
#divide sentences by tokenizer
def divide_sentences_review(train,tokenizer):
    sentences=[]
    num_reviews=len(train['review'])
    print 'divide reviews into sentences' #word2vec requires input to be a set of sentences as lits of words.
    for i in range(num_reviews):
        sentences += article_to_sentences(train['review'][i],tokenizer)
        if((i+1)%1000 == 0):
            print "Review %d th" % (i+1)
    return sentences

In [ ]:
def divide_sentences(train):
    sentences=[]
    num_articles=len(train)
    print 'divide articles into sentences...' #word2vec requires input to be a set of sentences as lits of words.
    for i in range(num_articles):
        for sentence in train[i].split('#'):
            if len(sentence)>1:
                sentences.append(sentence.split())
        if((i+1)%1000 == 0):
            print "Review %d th" % (i+1)
    return sentences

In [ ]:
def train_model(sentences,window_size,model_name):
    print "Training model..."
    model = word2vec.Word2Vec(sentences, workers=4, \
                size=300, min_count = 10, \
                window = window_size, sample = 0.003)
    model.save(model_name)
    return model

# Use model

In [ ]:
sentences=divide_sentences(train_text)

In [ ]:
model=train_model(sentences,2,'300features-2window-40mincount')

In [ ]:
for i,j in model.wv.vocab.keys()

In [ ]:
word_output=open('wordlist.txt','w')
word_output.write('\n'.join(model.wv.vocab.keys()))

In [ ]:
print model.wv.vocab.keys()[3]

In [99]:
for word,sim in model.most_similar('原住民'): #天安门，毛泽东，江泽民，习近平，曾志伟，中国，中华民国，劳动教养，警察，文革
    print '%s,%f' % (word,sim)

平埔族,0.764345
泰雅族,0.742261
住民,0.735763
土着,0.714229
西拉雅,0.710930
邹族,0.699622
阿美族,0.688832
原住,0.677996
布农族,0.677511
汉人,0.671237


In [ ]:
print model.doesnt_match("中国 台湾地区 中华民国 中华人民共和国 我国".split())

In [ ]:
print model.wv.most_similar(positive=['毛泽东', '反对'], negative=['邓小平'])[0][0]

In [19]:
model.wv.vocab['阿基米德'].index

18337

In [ ]:
model.wv.syn0[model.wv.vocab['中国'].index]

In [ ]:
model.score(["I don't like this movie".split()])

# Word to Phrases

In [ ]:
import gensim
phrases_2 = gensim.models.Phrases(sentences)

In [ ]:
test = list(phrases_2[sentences[0:2]])
print test

In [ ]:
phrases_3= gensim.models.Phrases(phrases_2[sentences])

In [ ]:
test2=list(phrases_3[test])
print test2

# Doc2Vec

In [144]:
laojiao = open("weibo_2016_tokens.txt", "r").readlines()

In [ ]:
word2vec_weight=pd.DataFrame(model.wv.syn0norm)

In [4]:
print model.wv.index2word[1079]

警察


## Weighting -- Average

In [ ]:
num=len(model.wv.vocab.keys())
laojiao_vec=[]
for i in range(len(laojiao)):
    case_vec=[0]*num
    line=laojiao[i].strip()
    words=line.split()
    length=len(words)
    for word in set(words):
        tf=words.count(word) #absolute word freq
        tf=round(float(tf)/length,2) #relative word freq
        if word in model.wv.vocab.keys():
            case_vec[model.wv.vocab[word].index]=tf
    case_vec=pd.Series(case_vec)
    output_vec=case_vec.dot(word2vec_weight).tolist()
    output_vec=[round(j,2) for j in output_vec]
    laojiao_vec.append(output_vec)
    if i % 100 ==0:
        print "%d Lines Have Been Vectorized" % i

## Weighting -- Down weighting frequent words (Pre-trained)

In [ ]:
Total=0
for word in model.wv.vocab.keys():
    Total+=model.wv.vocab[word].count

In [ ]:
vocab_num=len(model.wv.vocab.keys())
laojiao_vec=[]
a=0.001
iter_num=len(laojiao)
for i in range(iter_num):
    case_vec=[0]*vocab_num
    line=laojiao[i].strip()
    words=line.split()
    for word in set(words):
        tf=float("{0:.1f}".format(words.count(word))) #absolute word freq in doc
        if word in model.wv.vocab.keys():
            word_freq=model.wv.vocab[word].count/float(Total)
            case_vec[model.wv.vocab[word].index]=a/(a+tf*word_freq)
    case_vec=pd.Series(case_vec)
    output_vec=case_vec.dot(word2vec_weight).tolist()
    output_vec=[round(j,2) for j in output_vec]
    laojiao_vec.append(output_vec)
    if i % 100 ==0:
        print "%d Lines Have Been Vectorized" % i

## Weighting -- Down weighting frequent words (Testset)

In [145]:
#Use dense matrix representation to save storage
vocab_num=len(model.wv.vocab.keys())
dense_vec=[]
iter_num=len(laojiao)

print "Setup dense matrix of vocab"
for i in range(iter_num):
    line_vec=[]
    case_vec=[0]*vocab_num
    line=laojiao[i].strip()
    words=line.split()
    for word in words:
        if word in model.wv.vocab.keys():
            line_vec.append(model.wv.vocab[word].index)
    dense_vec.append(line_vec)
    if i % 100 ==0:
        print "%d Lines Have Been Vectorized" % i


Setup dense matrix of vocab
0 Lines Have Been Vectorized
100 Lines Have Been Vectorized
200 Lines Have Been Vectorized
300 Lines Have Been Vectorized
400 Lines Have Been Vectorized
500 Lines Have Been Vectorized
600 Lines Have Been Vectorized
700 Lines Have Been Vectorized
800 Lines Have Been Vectorized
900 Lines Have Been Vectorized
1000 Lines Have Been Vectorized
1100 Lines Have Been Vectorized
1200 Lines Have Been Vectorized
1300 Lines Have Been Vectorized
1400 Lines Have Been Vectorized
1500 Lines Have Been Vectorized
1600 Lines Have Been Vectorized
1700 Lines Have Been Vectorized
1800 Lines Have Been Vectorized
1900 Lines Have Been Vectorized
2000 Lines Have Been Vectorized
2100 Lines Have Been Vectorized
2200 Lines Have Been Vectorized
2300 Lines Have Been Vectorized
2400 Lines Have Been Vectorized
2500 Lines Have Been Vectorized
2600 Lines Have Been Vectorized
2700 Lines Have Been Vectorized
2800 Lines Have Been Vectorized
2900 Lines Have Been Vectorized
3000 Lines Have Been Vec

In [146]:
print "Calculate frequencies of vocab in whole Test-set"
dic={}
flat_list=[i for j in dense_vec for i in j]
Total=len(flat_list)
for word in set(flat_list):
    dic[word]=flat_list.count(word)/float(Total)

print "Done! Have updated %d Words' Frequencies." % len(dic.keys())

Calculate frequencies of vocab in whole Test-set
Done! Have updated 16983 Words' Frequencies.


In [147]:
len(dic.keys())

16983

In [103]:
i

7581

In [148]:
for k in dic.keys()[0:10]:
    print model.wv.index2word[k],round(dic[k],5)

中国 0.00256
一个 0.01044
香港 0.00057
台湾 0.00016
日本 0.00044
美国 0.00072
开始 0.00107
可以 0.00604
使用 0.00056
成为 0.00069


In [149]:
vocab_num=len(model.wv.vocab.keys())
word2vec_weight=pd.DataFrame(model.wv.syn0)
laojiao_vec=[]
a=0.001
iter_num=len(laojiao)
k=1
for case in dense_vec:
    case_vec=[0]*vocab_num
    for word in set(case):
        tf=float("{0:.1f}".format(case.count(word))) #absolute word freq in doc
        word_freq=dic[word]
        case_vec[int(word)]=tf*a/(a+word_freq)
    case_vec=pd.Series(case_vec)
    output_vec=case_vec.dot(word2vec_weight).tolist()
    output_vec=[round(j,2) for j in output_vec]
    laojiao_vec.append(output_vec)
    if k % 100 ==0:
        print "%d Lines Have Been Vectorized" % k
    k+=1

100 Lines Have Been Vectorized
200 Lines Have Been Vectorized
300 Lines Have Been Vectorized
400 Lines Have Been Vectorized
500 Lines Have Been Vectorized
600 Lines Have Been Vectorized
700 Lines Have Been Vectorized
800 Lines Have Been Vectorized
900 Lines Have Been Vectorized
1000 Lines Have Been Vectorized
1100 Lines Have Been Vectorized
1200 Lines Have Been Vectorized
1300 Lines Have Been Vectorized
1400 Lines Have Been Vectorized
1500 Lines Have Been Vectorized
1600 Lines Have Been Vectorized
1700 Lines Have Been Vectorized
1800 Lines Have Been Vectorized
1900 Lines Have Been Vectorized
2000 Lines Have Been Vectorized
2100 Lines Have Been Vectorized
2200 Lines Have Been Vectorized
2300 Lines Have Been Vectorized
2400 Lines Have Been Vectorized
2500 Lines Have Been Vectorized
2600 Lines Have Been Vectorized
2700 Lines Have Been Vectorized
2800 Lines Have Been Vectorized
2900 Lines Have Been Vectorized
3000 Lines Have Been Vectorized
3100 Lines Have Been Vectorized
3200 Lines Have B

## PCA (features reduction)

In [273]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(laojiao_vec)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [151]:
pca.explained_variance_ratio_.sum()

0.86835468807682525

In [274]:
laojiao_svd=pca.transform(laojiao_vec)

In [122]:
len(laojiao_svd)

12863

In [153]:
laojiao_svd=[i[1:] for i in laojiao_svd]

In [154]:
from sklearn.metrics.pairwise import cosine_similarity as sim
pairwise_similarity=sim(laojiao_svd)

In [53]:
n=2
pairwise_similarity[n][n+1:].argmax()

1

In [272]:
j+=1
k=pairwise_similarity[j].argsort()[-2]
print laojiao_svd[j]
print laojiao_svd[k]

[ 0.77744923 -0.17128495  1.99389749 -1.17916814 -0.94547904  1.12345079
  0.08958934 -0.5698971  -0.27551292 -0.24020523  0.71446441 -1.66644431
 -1.52628525 -0.32312708 -0.10529647 -0.9919811  -1.12826179 -0.00907144
  0.17191859]
[ 0.77744923 -0.17128495  1.99389749 -1.17916814 -0.94547904  1.12345079
  0.08958934 -0.5698971  -0.27551292 -0.24020523  0.71446441 -1.66644431
 -1.52628525 -0.32312708 -0.10529647 -0.9919811  -1.12826179 -0.00907144
  0.17191859]


In [162]:
n+=1
print laojiao[n]
print laojiao[pairwise_similarity[n].argsort()[-2]]

哥们 小伙伴 惊呆 

哥们 小伙伴 惊呆 



# Sentences Clustering

In [25]:
from sklearn.cluster import KMeans

In [275]:
kmeans = KMeans(n_clusters=10, random_state=0)
kmeans.fit(laojiao_svd)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [126]:
len(laojiao_svd)

7581

In [276]:
for i in range(10):
    print i,len(numpy.array(laojiao)[kmeans.labels_==i])

0 5060
1 811
2 166
3 69
4 136
5 167
6 2772
7 414
8 290
9 115


In [305]:
j=0
for i in range(5):
    boolean=kmeans.labels_==j
    l=len(numpy.array(laojiao)[boolean])
    print numpy.array(laojiao)[boolean][numpy.random.randint(l)]

太强大 穿透力 花心 给力 

哈哈 太有 整人 大师 给力 威武 来看 看吧 

特等奖 #崔晓静 1234 哈哈哈哈 哈哈 

奇葩 夫妻 欢乐 节操 一地 分享 大家 赶紧 围观 哈哈 

#络上 男女 #介绍 #微博 #哥靓 ##信上 放电 抛媚 

